In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import urllib.parse
from copy import copy
import requests
import re
import json
import urllib.request

## Scrape event information for a driver

In [2]:
def events_for_a_driver(my_url):
    
    # Load races on url
    raw_html = requests.get(my_url).content
    soup_doc = BeautifulSoup(raw_html, "html.parser")
    events = soup_doc.find_all("div", class_="profile-start-line d-flex justify-content-start w-100 lh-120 py-1 flex-wrap")
    races = []
    
    # Find all WRC races
    events = [e for e in events if e.span.text == ' WRC']

    # List out all informations
    
    for e in events:
        race = {}
        race['Driver'] = soup_doc.find("h4").a.text.strip()
        race['Name'] = e.find("a", title="Show entry info and stats").text
        URL = e.find("a", title="Show entry info and stats")['href']
        race['URL'] = URL
    # Check the URL
        print(URL)
        id1 = re.findall("/entryinfo/(\d+)",URL)[0]
        id2 = re.findall("/entryinfo/.+/(\d+)",URL)[0]
        Year = re.findall("/entryinfo/.+-(\d\d\d\d)/\d+/",URL)[0]
        race['Year'] = Year
        
        
        try:
            rank = e.find("span", class_="fs-09 font-weight-normal text-secondary").text
            ra = re.findall("(\d).",rank)[0]
            race['Race_Result'] = ra
            race['DNG_Reason'] = "Null"
        except:
            ra = e.find("div", class_="profile-start-ret text-danger font-weight-bold")
            race['Race_Result'] = "DNF"
            race['DNG_Reason'] = ra
        
        race['Php_url'] = "https://www.ewrc-results.com/inc/json_driverinfo.php?event="+id1+"&entry="+id2+"&sct=&cat=&lg=en"
        
        print("Php found" + race['Php_url'])
        
        x = urllib.request.urlopen(race['Php_url'])
        raw_data = x.read()
        encoding = x.info().get_content_charset('utf8')  # JSON default
        ssresult = json.loads(raw_data.decode(encoding))
        print(ssresult, len(ssresult['stages'][0][1]))

        
        for idx, pos in enumerate(ssresult['pos'][0][1]):
            
            stage = race.copy()
            
            stage['p_overall'] = pos      
            stage['stage'] = ssresult['stages'][0][1][idx]
            stage['ssname'] = ssresult['sslist'][idx]
            races.append(stage)
            # print(ssresult['pos'][0][1])
        
        
        print("Php info scraped.")
        
        
    return races



In [3]:
races = events_for_a_driver('https://www.ewrc-results.com/profile/2765-elfyn-evans/')

/entryinfo/79247-rally-sweden-2023/3608292/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=79247&entry=3608292&sct=&cat=&lg=en
{'pos': [['Overall', [3, 5, 7, 6, 4, 4, 4, 4, 4, 5, 5, 6, 5, 5, 5, 5, 5, 5]]], 'stages': [['Stage results', [3, 6, 7, 6, 5, 2, 4, 5, 4, 5, 6, 14, 5, 6, 6, 7, 7, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18'], 'colors': ['#e2431e', '#f1ca3a']} 18
Php info scraped.
/entryinfo/77787-rallye-automobile-monte-carlo-2023/3560668/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=77787&entry=3560668&sct=&cat=&lg=en
{'pos': [['Overall', [2, 2, 2, 2, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4]]], 'stages': [['Stage results', [2, 2, 2, 2, 22, 1, 2, 7, 2, 2, 2, 2, 3, 3, 2, 6, 7, 3]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18'], 'col

{'pos': [['Overall', [1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]]], 'stages': [['Stage results', [1, 1, 1, 2, 2, 2, 2, 5, 3, 3, 3, 4, 3, 3, 5, 6, 3]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17'], 'colors': ['#e2431e', '#f1ca3a']} 17
Php info scraped.
/entryinfo/68115-secto-rally-finland-2021/3118315/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=68115&entry=3118315&sct=&cat=&lg=en
{'pos': [['Overall', [7, 3, 4, 3, 5, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [7, 3, 5, 4, 5, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 19
Php info scraped.
/entryinfo/71264-eko-acropolis-rally-of-gods-2021/3096438/
Php foundhttps://www.ewrc-results.com/inc/json_dri

{'pos': [['Overall', [1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [1, 2, 1, 2, 1, 1, 1, 4, 6]]], 'sslist': ['SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS11'], 'colors': ['#e2431e', '#f1ca3a']} 9
Php info scraped.
/entryinfo/59972-rallye-automobile-monte-carlo-2020/2465687/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=59972&entry=2465687&sct=&cat=&lg=en
{'pos': [['Overall', [3, 4, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 3]]], 'stages': [['Stage results', [3, 4, 1, 1, 1, 3, 2, 3, 3, 1, 3, 2, 2, 2, 4, 4]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16'], 'colors': ['#e2431e', '#f1ca3a']} 16
Php info scraped.
/entryinfo/54472-rallyracc-catalunya-costa-daurada-2019/2392106/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=54472&entry=2392106&sct=&cat=&lg=en
{'pos': [['Overall', [6, 7, 4, 5, 6, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]]], 'stages'

{'pos': [['Overall', [20, 9, 5, 7, 4, 4, 4, 5, 6, 7, 13, 30, 28, 31, 34, 28, 25, 25]]], 'stages': [['Stage results', [20, 7, 4, 7, 4, 4, 5, 9, 7, 10, 53, 53, 45, 46, 46, 8, 8, 8]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18'], 'colors': ['#e2431e', '#f1ca3a']} 18
Php info scraped.
/entryinfo/44262-neste-rally-finland-2018/1880500/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=44262&entry=1880500&sct=&cat=&lg=en
{'pos': [['Overall', [9, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7]]], 'stages': [['Stage results', [9, 6, 7, 7, 7, 8, 9, 8, 8, 10, 9, 7, 6, 9, 6, 8, 8, 5, 10, 7, 8, 5, 7]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23'], 'colors': ['#e2431e', '#f1ca3a']} 23
Php info scraped.
/entryinfo/44261-rall

{'pos': [['Overall', [4, 10, 9, 33, 39, 40, 39, 39, 41, 38, 37, 30, 31, 30, 26, 27, 27, 26]]], 'stages': [['Stage results', [4, 10, 7, 46, 43, 40, 37, 33, 33, 7, 11, 7, 9, 9, 8, 7, 11, 9, 7]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 19
Php info scraped.
/entryinfo/36017-vodafone-rally-de-portugal-2017/1439520/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=36017&entry=1439520&sct=&cat=&lg=en
{'pos': [['Overall', [4, 8, 5, 10, 9, 9, 6, 6, 6, 5, 7, 7, 7, 6, 6, 6, 6, 6, 6]]], 'stages': [['Stage results', [4, 8, 2, 10, 5, 2, 4, 6, 2, 2, 24, 11, 10, 11, 8, 6, 11, 3, 3]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 19
Php info scraped.
/entryinfo/36016-ypf-rally-argentina-2017/143

{'pos': [['Overall', [13, 8, 5, 5, 5, 5, 5, 5, 5, 4, 3, 3]]], 'stages': [['Stage results', [13, 8, 3, 5, 3, 5, 7, 7, 2, 6, 9, 12]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12'], 'colors': ['#e2431e', '#f1ca3a']} 12
Php info scraped.
/entryinfo/19121-rally-guanajuato-mexico-2015/767995/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=19121&entry=767995&sct=&cat=&lg=en
{'pos': [['Overall', [10, 12, 8, 6, 5, 6, 6, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]], 'stages': [['Stage results', [10, 11, 9, 3, 6, 11, 7, 5, 5, 7, 5, 6, 5, 11, 4, 5, 6, 8, 7, 16, 8]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21'], 'colors': ['#e2431e', '#f1ca3a']} 21
Php info scraped.
/entryinfo/19120-rally-sweden-2015/743377/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=19120&entry=743377&sct=&cat=

{'pos': [['Overall', [14, 9, 5, 5, 6, 6, 7, 7, 5, 5, 6, 6, 7, 6]]], 'stages': [['Stage results', [14, 7, 2, 9, 8, 9, 9, 8, 7, 7, 4, 7, 8, 7]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS15'], 'colors': ['#e2431e', '#f1ca3a']} 14
Php info scraped.
/entryinfo/6537-rally-italia-sardegna-2013/343885/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=6537&entry=343885&sct=&cat=&lg=en
{'pos': [['Overall', [10, 11, 10, 10, 10, 10, 10, 9, 9, 8, 7, 8, 7, 6, 6, 6]]], 'stages': [['Stage results', [10, 12, 11, 11, 9, 10, 10, 8, 11, 8, 7, 10, 9, 8, 9, 9]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16'], 'colors': ['#e2431e', '#f1ca3a']} 16
Php info scraped.
/entryinfo/2373-wales-rally-gb-2011/132864/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=2373&entry=132864&sct=&cat=&lg=en
{'pos': [['Overall', [27, 31, 26

In [4]:
df = pd.DataFrame(races)
df

Driver                     Name  \
0     Elfyn Evans             Rally Sweden   
1     Elfyn Evans             Rally Sweden   
2     Elfyn Evans             Rally Sweden   
3     Elfyn Evans             Rally Sweden   
4     Elfyn Evans             Rally Sweden   
...           ...                      ...   
1769  Elfyn Evans  Neste Oil Rally Finland   
1770  Elfyn Evans  Neste Oil Rally Finland   
1771  Elfyn Evans  Neste Oil Rally Finland   
1772  Elfyn Evans  Neste Oil Rally Finland   
1773  Elfyn Evans  Neste Oil Rally Finland   

                                                    URL  Year Race_Result  \
0           /entryinfo/79247-rally-sweden-2023/3608292/  2023           5   
1           /entryinfo/79247-rally-sweden-2023/3608292/  2023           5   
2           /entryinfo/79247-rally-sweden-2023/3608292/  2023           5   
3           /entryinfo/79247-rally-sweden-2023/3608292/  2023           5   
4           /entryinfo/79247-rally-sweden-2023/3608292/  2023           5   
...                                                 ...   ...         ...   
1769  /entryinfo/2368-neste-oil-rally-finland-2011/1...  2011         DNF   
1770  /entryinfo/2368-neste-oil-rally-finland-2011/1...  2011         DNF   
1771  /entryinfo/2368-neste-oil-rally-finland-2011/1...  2011         DNF   
1772  /entryinfo/2368-neste-oil-rally-finland-2011/1...  2011         DNF   
1773  /entryinfo/2368-neste-oil-rally-finland-2011/1...  2011         DNF   

                                             DNG_Reason  \
0                                                  Null   
1                                                  Null   
2                                                  Null   
3                                                  Null   
4                                                  Null   
...                                                 ...   
1769  [\r\n                                Mechanica...   
1770  [\r\n                                Mechanica...   
1771  [\r\n                                Mechanica...   
1772  [\r\n                                Mechanica...   
1773  [\r\n                                Mechanica...   

                                                Php_url  p_overall  stage  \
0     https://www.ewrc-results.com/inc/json_driverin...          3      3   
1     https://www.ewrc-results.com/inc/json_driverin...          5      6   
2     https://www.ewrc-results.com/inc/json_driverin...          7      7   
3     https://www.ewrc-results.com/inc/json_driverin...          6      6   
4     https://www.ewrc-results.com/inc/json_driverin...          4      5   
...                                                 ...        ...    ...   
1769  https://www.ewrc-results.com/inc/json_driverin...         64     55   
1770  https://www.ewrc-results.com/inc/json_driverin...         63     55   
1771  https://www.ewrc-results.com/inc/json_driverin...         55     51   
1772  https://www.ewrc-results.com/inc/json_driverin...         54     68   
1773  https://www.ewrc-results.com/inc/json_driverin...         51     60   

     ssname  
0       SS1  
1       SS2  
2       SS3  
3       SS4  
4       SS5  
...     ...  
1769   SS15  
1770   SS16  
1771   SS17  
1772   SS18  
1773   SS19  

[1774 rows x 10 columns]

In [5]:
df.to_csv('Elfyn_Evans.csv')